## Production rate analysis of PUNDs
Author: Robert Ietswaart  
Date: 20240204  
License: BSD2.  
Load modules j3dl and activate virtual environment using j4RNAdecay on O2.  
Python v3.7.4

Source: `KP_20230427.ipynb`  
For RNA flow project. 

In [1]:
import os
import re
import copy
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import logging
import argparse
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
import seaborn as sns


# from __init__ import __version__
from __init__ import default_logger_format, default_date_format

In [5]:
# def main():
np.random.seed(12345)

parser = argparse.ArgumentParser(
    description='Estimating the fraction of transcripts that get nuclear degraded, ' 
                'through production rate estimates kp.')

args = parser.parse_args("")#EDIT: added "" as argument to run in ipynb instead of .py script

In [21]:
path = dict() 
path['h'] = dict()
path['m'] = dict()
path['h']['chr'] = os.path.join('/n', 'groups', 'churchman', 'bms36', '2021-05-16_T_U')
path['h']['nuc'] = os.path.join('/n', 'groups', 'churchman', 'bms36', '2021-04-07_T_U')
path['h']['cyto'] = path['h']['chr']
path['h']['poly'] = path['h']['chr']
path['h']['tot'] = path['h']['nuc']
path['m']['chr'] = os.path.join('/n', 'groups', 'churchman', 'bms36', '2021-04-05_R_S')
path['m']['nuc'] = os.path.join('/n', 'groups', 'churchman', 'bms36', '2020-10-06_G_H_fract')
path['m']['cyto'] = path['m']['nuc']
path['m']['poly'] = os.path.join('/n', 'groups', 'churchman', 'bms36', '2020-07-13_G_H')
path['m']['tot'] = path['m']['poly']

outpath = os.path.join('/n','groups','churchman','ri23','bseq','KP20240204')

# Add a logger specific to the project and processing stage
logger = logging.getLogger('KP')
log_file = os.path.join(outpath,'LogErr', 'KP_20240204.log')
formatter = logging.Formatter(default_logger_format,
                              datefmt=default_date_format)
log_handler = logging.FileHandler(log_file)
log_handler.setFormatter(formatter)
logger.addHandler(log_handler)

organisms = ['m','h']
org_map = {'m': 'mouse', 'h': 'human'}
org_reps = {'m': ['G','H','R','S'], 'h': ['T', 'U']}
org_red_reps = {'m': ['G_R','H_S'], 'h': ['T', 'U']}
reps = ['G','H','R','S','T','U']
red_r = {'G': 'G_R', 'H': 'H_S', 'R': 'G_R', 'S': 'H_S', 'T': 'T', 'U': 'U'}
fracs = ['chr', 'nuc', 'cyto', 'poly', 'tot']
k_bound_lo = 1e-4 #unit: min^-1: 1 per 7 days
k_bound_hi = 1e4 #unit: min^-1: 1 per 6 ms

RATE_TYPE = ['half_life_','k_']
rt = RATE_TYPE[1] #ML model dependent variable: rates
TC_TYPES = ['top1000','bottom500']#BM model turnover method to estimate TC
OUT_TYPES = ['.Mean', '.MAP', '.0.025.quantile', '.0.975.quantile']
SUFFICES = {'top1000': ['genes.turnover'], 'bottom500': ['', '_below']}

Timescales = ['chr',
              'chr_release',
              'nucdeg',
              'nucexp',
              'nuc',
              'cyto',
              'poly_entry',
              'whole_cell']
Timescales = [rt + ts for ts in Timescales]

kp_fr = 'chr'#'chromatin' 
kp_ot = OUT_TYPES[1]#

ptc_genes = dict() #List with protein-coding genes ENS IDs
ptc_genes['h'] = os.path.join('/n', 'groups', 'churchman', 'mc348','TimelapseSeq','SeqFiles',
                              'K562_ensGRCh38_MTmod_ptc_list.txt')
ptc_genes['m'] = os.path.join('/n', 'groups', 'churchman', 'mc348','TimelapseSeq','SeqFiles',
                              'NIH3T3_mm10_MTmod_ptc_list.txt')

logger.info('Load Rates and GRAND-SLAM outputs')
B = dict()          #Bayes fits file
K = dict()          #Bayes Factor
GS = dict()         #GRAND-SLAM
PTC = dict()

path_b = dict()
path_b['h'] = os.path.join('/n','groups','churchman','ri23','bseq','Bayes20240120_K562')#'Bayes20240119_K562'#'Bayes20231201_K562')
path_b['m'] = os.path.join('/n','groups','churchman','ri23','bseq','Bayes20240120_3T3') #'Bayes20231201_3T3')
path_k = os.path.join('/n','groups','churchman','ri23','bseq','BayesFactor20240112')

for o in organisms:
    PTC[o] = pd.read_csv(os.path.join(ptc_genes[o]))
    GS[o] = dict()
    
    filename_b = 'Bayes_Rates_20240120_'+ org_map[o] + '_final.tsv'#'Bayes_Rates_20240119_'+ org_map[o] + '.tsv' #20231201
    filename_k = 'Bayes_factor_20240112_' + org_map[o] + '_final.tsv'

    B[o] = pd.read_csv(os.path.join(path_b[o], filename_b), sep='\t')
    K[o] = pd.read_csv(os.path.join(path_k, filename_k), sep='\t')
    
    for r in reps:
        for fr in fracs:
            for tc in TC_TYPES:
                path_gs = os.path.join(path[o][fr],'GS_2023',r+'-'+fr,re.sub(r'\d', '', tc))
                filename_gs = r + '_' + fr + '.tsv'
                if os.path.exists(os.path.join(path_gs, filename_gs)):
                    GS[o][r+fr+tc]= pd.read_csv(os.path.join(path_gs, filename_gs), sep='\t')
                    #filter for protein-coding genes
                    GS[o][r+fr+tc] = GS[o][r+fr+tc][GS[o][r+fr+tc]['Gene'].isin(PTC[o]['Gene'])]
                    GS[o][r+fr+tc].sort_values(by='Gene',inplace=True, ignore_index=True) 

INFO: [2024-02-07 21:06:00] KP - Load Rates and GRAND-SLAM outputs


### Preprocess rates: dependent variables in model
- preprocess nucexp: nucexp_from_chr or nucexp_from_nucdeg depending on nucdeg no or yes 
- clip to domain bounds

In [23]:
tct = TC_TYPES[0]

C = copy.deepcopy(B)
for o in organisms:
    C[o] = C[o].merge(K[o], on='Gene', how='outer', suffixes=('', '_bf'))
    
    for r in org_reps[o]:
        if r + kp_fr + tct in GS[o].keys():
            rr = red_r[r]
            n_reads = GS[o][r+kp_fr+tct][r+'1 Readcount'].sum()
            GS[o][r+kp_fr+tct][rr+'.RPKM'+kp_fr] = GS[o][r+kp_fr+tct][r+'1 Readcount']/ \
                (GS[o][r+kp_fr+tct]['Length'] * 1e-3 * n_reads * 1e-6)
            C[o] = C[o].merge(GS[o][r+kp_fr+tct], on='Gene', how='inner', suffixes=('', '_gs_'+rr))
            C[o][rr+'.kp'] = C[o][rr+'.RPKM'+kp_fr] * C[o][rr+'.'+rt+kp_fr+kp_ot]
            
    logger.info('%s all genes: %d' % (org_map[o], len(C[o])))

    for ts in Timescales:
        for ot in OUT_TYPES:
            for rr in org_red_reps[o]:
                if ts == rt + 'chr_release':
                    C[o][rr+'.'+ts+ot] = copy.deepcopy(C[o][rr+'.'+ts+'_from_nucdeg.MAP'].where(
                        C[o]['PUND'], C[o][rr+'.'+rt+'chr'+ot]))
                if ts == rt + 'nucdeg':#only for nucdeg genes according to Bayes Factor
                    C[o][rr+'.'+ts+ot] = copy.deepcopy(C[o][rr+'.'+ts+'.MAP'].where(C[o]['PUND'], k_bound_lo))
                if ts == rt + 'nucexp':
                    C[o][rr+'.'+ts+ot] = copy.deepcopy(C[o][rr+'.'+ts+'_from_nucdeg.MAP'].where(
                        C[o]['PUND'], C[o][rr+'.'+ts+'_from_nucres'+ot]))

INFO: [2024-02-07 21:06:53] KP - mouse all genes: 15277
INFO: [2024-02-07 21:06:53] KP - human all genes: 14987


### Calculate fraction of degraded transcripts

In [25]:
for o in organisms:
    for rr in org_red_reps[o]:
        C[o][rr+'.frac_ND'] = C[o][rr+'.k_nucdeg'+kp_ot] / \
                              (C[o][rr+'.k_nucdeg'+kp_ot] + C[o][rr+'.k_chr_release'+kp_ot]) \
        * (C[o][rr+'.k_chr_release'+kp_ot] + C[o][rr+'.k_nucdeg'+kp_ot] + C[o][rr+'.k_nucexp'+kp_ot]) \
            / (C[o][rr+'.k_nucdeg'+kp_ot] + C[o][rr+'.k_nucexp'+kp_ot])
        C[o][rr+'.kp_ND'] = C[o][rr+'.frac_ND'] * C[o][rr+'.kp']

## Standard PUNDs criteria

In [26]:
logger.info('kp derived from compartment: %s with %s' % (kp_fr, kp_ot))


for o in organisms:
    C[o] = C[o][~C[o]['PUND'].isna()]
    logger.info('%s all genes: %d' % (org_map[o], len(C[o])))
    
    global_kp_nd = C[o][C[o]['PUND']][[rr+'.kp_ND' for rr in org_red_reps[o]]].sum().sum()
#     logger.info(global_kp_nd)
    frac_nd_of_all = global_kp_nd / C[o][[rr+'.kp' for rr in org_red_reps[o]]].sum().sum()
    logger.info('%s fraction of transcripts nucdegraded over all produced protein-coding transcripts: %s' % 
                (o,frac_nd_of_all))
    frac_nd_of_PUNDs = global_kp_nd / C[o][C[o]['PUND']][[rr+'.kp' for rr in org_red_reps[o]]].sum().sum()
    logger.info('%s fraction of transcripts nucdegraded over all produced PUND transcripts: %s' % 
                (o,frac_nd_of_PUNDs))  
   

INFO: [2024-02-07 21:07:06] KP - kp derived from compartment: chr with .MAP
INFO: [2024-02-07 21:07:06] KP - mouse all genes: 11667
INFO: [2024-02-07 21:07:06] KP - m fraction of transcripts nucdegraded over all produced protein-coding transcripts: 0.10491634856471654
INFO: [2024-02-07 21:07:06] KP - m fraction of transcripts nucdegraded over all produced PUND transcripts: 0.7063311697131196
INFO: [2024-02-07 21:07:06] KP - human all genes: 11409
INFO: [2024-02-07 21:07:06] KP - h fraction of transcripts nucdegraded over all produced protein-coding transcripts: 0.05558735840023887
INFO: [2024-02-07 21:07:06] KP - h fraction of transcripts nucdegraded over all produced PUND transcripts: 0.5175728183358114


### Scraps

In [27]:
for o in organisms:
    logger.info(o)

    for rr in org_red_reps[o]:
        logger.info(rr)
        logger.info(C[o][C[o][rr+'.kp']==C[o][rr+'.kp'].max()])
        logger.info(C[o][C[o][rr+'.kp']==C[o][rr+'.kp'].min()])
        logger.info(C[o][rr+'.kp'].median())

INFO: [2024-02-07 21:07:23] KP - m
INFO: [2024-02-07 21:07:23] KP - G_R
INFO: [2024-02-07 21:07:23] KP -                     Gene  Symbol  G_R.half_life_chr.Mean  \
3317  ENSMUSG00000022884  Eif4a2               16.615057   

      G_R.half_life_chr.MAP  G_R.half_life_chr.0.025.quantile  \
3317              16.680415                         16.076475   

      G_R.half_life_chr.0.975.quantile  G_R.half_life_nuc.Mean  \
3317                         17.307042                16.20718   

      G_R.half_life_nuc.MAP  G_R.half_life_nuc.0.025.quantile  \
3317              16.076475                         15.494402   

      G_R.half_life_nuc.0.975.quantile  G_R.half_life_nucexp_from_nucres.Mean  \
3317                          16.99084                                  1.569   

      G_R.half_life_nucexp_from_nucres.MAP  \
3317                              1.858891   

      G_R.half_life_nucexp_from_nucres.0.025.quantile  \
3317                                         0.872822   

      G_

INFO: [2024-02-07 21:07:24] KP -                     Gene Symbol  G_R.half_life_chr.Mean  \
4364  ENSMUSG00000025888  Casp1                660.3921   

      G_R.half_life_chr.MAP  G_R.half_life_chr.0.025.quantile  \
4364            6931.471806                        167.189055   

      G_R.half_life_chr.0.975.quantile  G_R.half_life_nuc.Mean  \
4364                       6092.133406               79.647959   

      G_R.half_life_nuc.MAP  G_R.half_life_nuc.0.025.quantile  \
4364              103.51413                         36.859648   

      G_R.half_life_nuc.0.975.quantile  G_R.half_life_nucexp_from_nucres.Mean  \
4364                         275.05786                              51.453871   

      G_R.half_life_nucexp_from_nucres.MAP  \
4364                             82.966647   

      G_R.half_life_nucexp_from_nucres.0.025.quantile  \
4364                                         20.43125   

      G_R.half_life_nucexp_from_nucres.0.975.quantile  \
4364                     

INFO: [2024-02-07 21:07:24] KP - 0.13049896913943895
INFO: [2024-02-07 21:07:24] KP - H_S
INFO: [2024-02-07 21:07:24] KP -                      Gene     Symbol  G_R.half_life_chr.Mean  \
13444  ENSMUSG00000069309  Hist1h2an               25.882272   

       G_R.half_life_chr.MAP  G_R.half_life_chr.0.025.quantile  \
13444               25.96564                         24.119428   

       G_R.half_life_chr.0.975.quantile  G_R.half_life_nuc.Mean  \
13444                          27.95317               51.143548   

       G_R.half_life_nuc.MAP  G_R.half_life_nuc.0.025.quantile  \
13444              51.368316                         45.988292   

       G_R.half_life_nuc.0.975.quantile  \
13444                         56.329432   

       G_R.half_life_nucexp_from_nucres.Mean  \
13444                              39.267763   

       G_R.half_life_nucexp_from_nucres.MAP  \
13444                             39.681056   

       G_R.half_life_nucexp_from_nucres.0.025.quantile  \
13444     

INFO: [2024-02-07 21:07:25] KP -                     Gene  Symbol  G_R.half_life_chr.Mean  \
1014  ENSMUSG00000008496  Pou2f2                47.79031   

      G_R.half_life_chr.MAP  G_R.half_life_chr.0.025.quantile  \
1014              58.445542                         23.678762   

      G_R.half_life_chr.0.975.quantile  G_R.half_life_nuc.Mean  \
1014                        131.553904               39.289106   

      G_R.half_life_nuc.MAP  G_R.half_life_nuc.0.025.quantile  \
1014              45.148079                         21.593297   

      G_R.half_life_nuc.0.975.quantile  G_R.half_life_nucexp_from_nucres.Mean  \
1014                         87.685456                              30.854481   

      G_R.half_life_nucexp_from_nucres.MAP  \
1014                             37.545613   

      G_R.half_life_nucexp_from_nucres.0.025.quantile  \
1014                                        15.782755   

      G_R.half_life_nucexp_from_nucres.0.975.quantile  \
1014                   

INFO: [2024-02-07 21:07:25] KP - 0.09246171420472943
INFO: [2024-02-07 21:07:25] KP - h
INFO: [2024-02-07 21:07:25] KP - T
INFO: [2024-02-07 21:07:25] KP -                   Gene Symbol  T.half_life_chr.Mean  T.half_life_chr.MAP  \
12469  ENSG00000196262   PPIA             13.008114            13.125104   

       T.half_life_chr.0.025.quantile  T.half_life_chr.0.975.quantile  \
12469                       12.418774                        13.61817   

       T.half_life_nuc.Mean  T.half_life_nuc.MAP  \
12469             38.056126            38.244343   

       T.half_life_nuc.0.025.quantile  T.half_life_nuc.0.975.quantile  \
12469                       36.186217                       39.681056   

       T.half_life_nucexp_from_nucres.Mean  \
12469                                  NaN   

       T.half_life_nucexp_from_nucres.MAP  \
12469                                 NaN   

       T.half_life_nucexp_from_nucres.0.025.quantile  \
12469                                            NaN

INFO: [2024-02-07 21:07:26] KP -                  Gene    Symbol  T.half_life_chr.Mean  T.half_life_chr.MAP  \
150   ENSG00000007402  CACNA2D2             30.670514            33.613300   
1890  ENSG00000100234     TIMP3            174.485797           204.783957   
4325  ENSG00000122035   RASL11A              1.109251             1.109251   
4916  ENSG00000128383  APOBEC3A             86.404960            92.672651   
5508  ENSG00000133636       NTS            273.741256           758.340936   
5512  ENSG00000133678   TMEM254             89.958485           107.402812   
9215  ENSG00000165496    RPL10L            124.056080           356.070645   

      T.half_life_chr.0.025.quantile  T.half_life_chr.0.975.quantile  \
150                        18.631803                       57.377732   
1890                       90.979507                      452.522602   
4325                       16.375661                       41.171741   
4916                       55.300284                  

INFO: [2024-02-07 21:07:26] KP - 0.17235357620724914
INFO: [2024-02-07 21:07:26] KP - U
INFO: [2024-02-07 21:07:26] KP -                  Gene Symbol  T.half_life_chr.Mean  T.half_life_chr.MAP  \
9853  ENSG00000168906  MAT2A             32.999179            32.999179   

      T.half_life_chr.0.025.quantile  T.half_life_chr.0.975.quantile  \
9853                       32.396279                         33.6133   

      T.half_life_nuc.Mean  T.half_life_nuc.MAP  \
9853             31.804394            31.804394   

      T.half_life_nuc.0.025.quantile  T.half_life_nuc.0.975.quantile  \
9853                       31.223322                       32.999179   

      T.half_life_nucexp_from_nucres.Mean  T.half_life_nucexp_from_nucres.MAP  \
9853                             3.679192                            3.886639   

      T.half_life_nucexp_from_nucres.0.025.quantile  \
9853                                       2.687906   

      T.half_life_nucexp_from_nucres.0.975.quantile  \
9853  

INFO: [2024-02-07 21:07:27] KP -                   Gene      Symbol  T.half_life_chr.Mean  T.half_life_chr.MAP  \
677    ENSG00000058866        DGKG                   NaN                  NaN   
1890   ENSG00000100234       TIMP3            174.485797           204.783957   
2191   ENSG00000102010         BMX                   NaN                  NaN   
2589   ENSG00000105357       MYH14            105.501545           122.200141   
3997   ENSG00000118508       RAB32             56.472487            67.735361   
4676   ENSG00000125775      SDCBP2                   NaN                  NaN   
7857   ENSG00000154930       ACSS1             51.625321            57.377732   
8151   ENSG00000158292      GPR153             94.461897            99.766243   
8767   ENSG00000163467       TSACC             84.914151            94.397305   
8826   ENSG00000163683      SMIM14                   NaN                  NaN   
9493   ENSG00000166924       NYAP1             30.081265            31.80439

INFO: [2024-02-07 21:07:27] KP - 0.1487810454223209


In [ ]:
#run with kp_ot == '.Mean'

In [18]:
logger.info('kp derived from compartment: %s with %s' % (kp_fr, kp_ot))


for o in organisms:
    C[o] = C[o][~C[o]['PUND'].isna()]
    logger.info('%s all genes: %d' % (org_map[o], len(C[o])))
    
    global_kp_nd = C[o][C[o]['PUND']][[rr+'.kp_ND' for rr in org_red_reps[o]]].sum().sum()
#     logger.info(global_kp_nd)
    frac_nd_of_all = global_kp_nd / C[o][[rr+'.kp' for rr in org_red_reps[o]]].sum().sum()
    logger.info('%s fraction of transcripts nucdegraded over all produced protein-coding transcripts: %s' % 
                (o,frac_nd_of_all))
    frac_nd_of_PUNDs = global_kp_nd / C[o][C[o]['PUND']][[rr+'.kp' for rr in org_red_reps[o]]].sum().sum()
    logger.info('%s fraction of transcripts nucdegraded over all produced PUND transcripts: %s' % 
                (o,frac_nd_of_PUNDs))  
   

INFO: [2024-02-07 21:01:14] KP - kp derived from compartment: chr with .Mean
INFO: [2024-02-07 21:01:14] KP - mouse all genes: 11667
INFO: [2024-02-07 21:01:14] KP - m fraction of transcripts nucdegraded over all produced protein-coding transcripts: 0.08338984207557446
INFO: [2024-02-07 21:01:14] KP - m fraction of transcripts nucdegraded over all produced PUND transcripts: 0.7145732008129881
INFO: [2024-02-07 21:01:14] KP - human all genes: 11409
INFO: [2024-02-07 21:01:14] KP - h fraction of transcripts nucdegraded over all produced protein-coding transcripts: 0.05317285421019376
INFO: [2024-02-07 21:01:14] KP - h fraction of transcripts nucdegraded over all produced PUND transcripts: 0.5189933125295704


In [20]:
for o in organisms:
    logger.info(o)

    for rr in org_red_reps[o]:
        logger.info(rr)
        logger.info(C[o][C[o][rr+'.kp']==C[o][rr+'.kp'].max()])
        logger.info(C[o][C[o][rr+'.kp']==C[o][rr+'.kp'].min()])
        logger.info(C[o][rr+'.kp'].median())

INFO: [2024-02-07 21:03:48] KP - m
INFO: [2024-02-07 21:03:48] KP - G_R
INFO: [2024-02-07 21:03:48] KP -                     Gene Symbol  G_R.half_life_chr.Mean  \
6122  ENSMUSG00000029552    Tes                0.248221   

      G_R.half_life_chr.MAP  G_R.half_life_chr.0.025.quantile  \
6122              72.920138                         71.587876   

      G_R.half_life_chr.0.975.quantile  G_R.half_life_nuc.Mean  \
6122                         74.277194               124.32486   

      G_R.half_life_nuc.MAP  G_R.half_life_nuc.0.025.quantile  \
6122             124.474306                        115.623918   

      G_R.half_life_nuc.0.975.quantile  G_R.half_life_nucexp_from_nucres.Mean  \
6122                        134.002144                              72.497521   

      G_R.half_life_nucexp_from_nucres.MAP  \
6122                             72.920138   

      G_R.half_life_nucexp_from_nucres.0.025.quantile  \
6122                                        64.090171   

      G_R.

INFO: [2024-02-07 21:03:49] KP -                      Gene Symbol  G_R.half_life_chr.Mean  \
11761  ENSMUSG00000051747    Ttn              797.517314   

       G_R.half_life_chr.MAP  G_R.half_life_chr.0.025.quantile  \
11761             963.759349                         405.12797   

       G_R.half_life_chr.0.975.quantile  G_R.half_life_nuc.Mean  \
11761                       2209.677861               36.239648   

       G_R.half_life_nuc.MAP  G_R.half_life_nuc.0.025.quantile  \
11761              38.956077                         23.678762   

       G_R.half_life_nuc.0.975.quantile  \
11761                         60.641146   

       G_R.half_life_nucexp_from_nucres.Mean  \
11761                              36.158036   

       G_R.half_life_nucexp_from_nucres.MAP  \
11761                             46.844142   

       G_R.half_life_nucexp_from_nucres.0.025.quantile  \
11761                                        17.307042   

       G_R.half_life_nucexp_from_nucres.0.975.qua

INFO: [2024-02-07 21:03:49] KP - 0.12771441049767635
INFO: [2024-02-07 21:03:49] KP - H_S
INFO: [2024-02-07 21:03:49] KP -                   Gene Symbol  G_R.half_life_chr.Mean  G_R.half_life_chr.MAP  \
98  ENSMUSG00000000740  Rpl13               69.188967              68.995928   

    G_R.half_life_chr.0.025.quantile  G_R.half_life_chr.0.975.quantile  \
98                         64.090171                         72.920138   

    G_R.half_life_nuc.Mean  G_R.half_life_nuc.MAP  \
98               153.99999             155.301558   

    G_R.half_life_nuc.0.025.quantile  G_R.half_life_nuc.0.975.quantile  \
98                        144.259286                        164.134484   

    G_R.half_life_nucexp_from_nucres.Mean  \
98                             111.567746   

    G_R.half_life_nucexp_from_nucres.MAP  \
98                            111.437579   

    G_R.half_life_nucexp_from_nucres.0.025.quantile  \
98                                        101.62291   

    G_R.half_life_nu

INFO: [2024-02-07 21:03:50] KP -                      Gene Symbol  G_R.half_life_chr.Mean  \
11761  ENSMUSG00000051747    Ttn              797.517314   

       G_R.half_life_chr.MAP  G_R.half_life_chr.0.025.quantile  \
11761             963.759349                         405.12797   

       G_R.half_life_chr.0.975.quantile  G_R.half_life_nuc.Mean  \
11761                       2209.677861               36.239648   

       G_R.half_life_nuc.MAP  G_R.half_life_nuc.0.025.quantile  \
11761              38.956077                         23.678762   

       G_R.half_life_nuc.0.975.quantile  \
11761                         60.641146   

       G_R.half_life_nucexp_from_nucres.Mean  \
11761                              36.158036   

       G_R.half_life_nucexp_from_nucres.MAP  \
11761                             46.844142   

       G_R.half_life_nucexp_from_nucres.0.025.quantile  \
11761                                        17.307042   

       G_R.half_life_nucexp_from_nucres.0.975.qua

INFO: [2024-02-07 21:03:50] KP - 0.08984593455321586
INFO: [2024-02-07 21:03:50] KP - h
INFO: [2024-02-07 21:03:50] KP - T
INFO: [2024-02-07 21:03:50] KP -                  Gene Symbol  T.half_life_chr.Mean  T.half_life_chr.MAP  \
5488  ENSG00000133316  WDR74              2.908284            101.62291   

      T.half_life_chr.0.025.quantile  T.half_life_chr.0.975.quantile  \
5488                       99.766243                       103.51413   

      T.half_life_nuc.Mean  T.half_life_nuc.MAP  \
5488            176.454781           176.698095   

      T.half_life_nuc.0.025.quantile  T.half_life_nuc.0.975.quantile  \
5488                      167.189055                      183.336056   

      T.half_life_nucexp_from_nucres.Mean  T.half_life_nucexp_from_nucres.MAP  \
5488                            99.694054                           99.766243   

      T.half_life_nucexp_from_nucres.0.025.quantile  \
5488                                      92.672651   

      T.half_life_nucexp_f

INFO: [2024-02-07 21:03:50] KP -                  Gene    Symbol  T.half_life_chr.Mean  T.half_life_chr.MAP  \
150   ENSG00000007402  CACNA2D2             30.670514            33.613300   
1890  ENSG00000100234     TIMP3            174.485797           204.783957   
4325  ENSG00000122035   RASL11A              1.109251             1.109251   
4916  ENSG00000128383  APOBEC3A             86.404960            92.672651   
5508  ENSG00000133636       NTS            273.741256           758.340936   
5512  ENSG00000133678   TMEM254             89.958485           107.402812   
9215  ENSG00000165496    RPL10L            124.056080           356.070645   

      T.half_life_chr.0.025.quantile  T.half_life_chr.0.975.quantile  \
150                        18.631803                       57.377732   
1890                       90.979507                      452.522602   
4325                       16.375661                       41.171741   
4916                       55.300284                  

INFO: [2024-02-07 21:03:51] KP - 0.17197080354801486
INFO: [2024-02-07 21:03:51] KP - U
INFO: [2024-02-07 21:03:51] KP -                  Gene  Symbol  T.half_life_chr.Mean  T.half_life_chr.MAP  \
2153  ENSG00000101639  CEP192             90.899681            90.979507   

      T.half_life_chr.0.025.quantile  T.half_life_chr.0.975.quantile  \
2153                       87.685456                       94.397305   

      T.half_life_nuc.Mean  T.half_life_nuc.MAP  \
2153             129.28557           129.150393   

      T.half_life_nuc.0.025.quantile  T.half_life_nuc.0.975.quantile  \
2153                      124.474306                      134.002144   

      T.half_life_nucexp_from_nucres.Mean  T.half_life_nucexp_from_nucres.MAP  \
2153                            54.481765                           55.300284   

      T.half_life_nucexp_from_nucres.0.025.quantile  \
2153                                      48.603921   

      T.half_life_nucexp_from_nucres.0.975.quantile  \
2153

INFO: [2024-02-07 21:03:51] KP -                   Gene      Symbol  T.half_life_chr.Mean  T.half_life_chr.MAP  \
677    ENSG00000058866        DGKG                   NaN                  NaN   
1890   ENSG00000100234       TIMP3            174.485797           204.783957   
2191   ENSG00000102010         BMX                   NaN                  NaN   
2589   ENSG00000105357       MYH14            105.501545           122.200141   
3997   ENSG00000118508       RAB32             56.472487            67.735361   
4676   ENSG00000125775      SDCBP2                   NaN                  NaN   
7857   ENSG00000154930       ACSS1             51.625321            57.377732   
8151   ENSG00000158292      GPR153             94.461897            99.766243   
8767   ENSG00000163467       TSACC             84.914151            94.397305   
8826   ENSG00000163683      SMIM14                   NaN                  NaN   
9493   ENSG00000166924       NYAP1             30.081265            31.80439

INFO: [2024-02-07 21:03:52] KP - 0.14887951020168688
